<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


# The async futures interface

In previous notebooks we showed that the distributed scheduler can be used to execute dask graphs, and so compute collections across multiple machines. However, the distributed scheduler also offers additional API that allow for finer control and asynchronous computation. In this notebook we delve into these features.

To begin, the `distributed` scheduler implements a superset of of the standard library [`concurrent.futures`](https://docs.python.org/3/library/concurrent.futures.html) API, allowing for asynchronous map-reduce like functionality. We can submit individual functions for evaluation with one set of inputs, or evaluated over a sequence of inputs with `submit()` and `map()`. Notice that the call returns immediately, giving one or more *futures*, whose status begins as "pending" and later becomes "finished". There is no blocking of the local python session.

Before you start, you should open the diagnostics dashboard so you can watch what the scheduler is doing.

In [ ]:
from pycon_utils import make_cluster
from dask.distributed import Client

In [ ]:
c = Client(make_cluster())
c

In [ ]:
# Define some toy functions for simulating work
import time

def inc(x):
    time.sleep(5)
    return x + 1

def dec(x):
    time.sleep(3)
    return x - 1

def add(x, y):
    time.sleep(7)
    return x + y

### `Client.submit`

`submit` takes a function and arguments, pushes these to the cluster, returning a *Future* representing the result to be computed. The function is passed to a worker process for evaluation. Note that this cell returns immediately, while computation may still be ongoing on the cluster.

In [ ]:
fut = c.submit(inc, 1)
fut

Looking at the `repr` of the future, `fut`, you can see it's status is "pending". This means that the computation the future represents hasn't finished yet. Since this is asynchronous, you can do other things while it computes. However, if you want to wait until the future has completed you can use the `wait` function. At this time the future's status will become "finished".

In [ ]:
# Block until `fut` has completed
wait(fut);

To retrieve a result you can use the `.result` method (which fetches for the one future), or `Client.gather` (which fetches for one or more futures). In both cases these methods will block until all requested futures have completed.

In [ ]:
# Retrieve the data using `.result()`
fut.result()

In [ ]:
# Retrieve the data using `Client.gather` - result is now already available
c.gather(fut)

Here we see an alternative way to execute work on the cluster: when you submit or map with the inputs as futures, the *computation moves to the data* rather than the other way around, and the client, in the local python session, need never see the intermediate values. This is similar to building the graph using delayed, and indeed, delayed can be used in conjunction with futures. Here we use the delayed object `total` from before.

In [ ]:
from dask import delayed

x = delayed(inc)(1)
y = delayed(dec)(2)
total = delayed(add)(x, y)

In [ ]:
# notice the difference from total.compute()
# notice that this cell completes immediately
fut = c.compute(total)
fut

In [ ]:
c.gather(fut)

### Exercise: Rebuild the above delayed computation using `Client.submit` instead

Solution:

In [ ]:
%load solutions/client_submit.py

The futures API offers a work submission style that can easily emulate the map/reduce paradigm (see `c.map()`) that may be familiar. The intermediate results, represented by futures, can be passed to new tasks without having to bring the data locally from the cluster, and new work can be assigned using the output of previous jobs that haven't begun yet.

Generally, any dask operation that is executed using `.compute()` can be submitted for asynchronous execution using `c.compute()` instead, and this applies to all collections. Here we create a `Bag`, do some operations on it, and call `Client.compute` on it. Since this is asynchronous we could continue to submit more work (perhaps based on the result of the calculation), or, as in the next cell, follow the progress of the computation. A similar progress-bar appears in the monitoring UI page.

In [ ]:
import dask.bag as db

res = (db.from_sequence(range(10))
         .map(inc)
         .filter(lambda x: x % 2 == 0)
         .sum())

res

In [ ]:
f = c.compute(res)
f

In [ ]:
# note that progress must be the last line of a cell
# in order to show up
progress(f)

In [ ]:
c.gather(f)

**Note well**: a future points to data being computed on the cluster or held in memory. To release the memory, all references to a future need to be deleted. Look at the scheduler dashboard to see the effect of the following.

In [ ]:
del f, fut

## Asynchronous computation
<img style="float: right;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Rosenbrock_function.svg/450px-Rosenbrock_function.svg.png" height=200 width=200>

One benefit of using the futures api is that you can have dynamic computations that adjust as things progress. Here we implement a simple naive search by looping through results as they come in, and submit new points to compute as others are still running.

Watching the [diagnostics dashboard](../../9002/status) as this runs you can see computations are being concurrently run while more are being submitted. This flexibility can be useful for parallel algorithms that require some level of synchronization.

In [ ]:
# a simple function with interesting minima

def rosenbrock(point):
    """Compute the rosenbrock function and return the point and result"""
    time.sleep(0.1)
    score = (1 - point[0])**2 + 2 * (point[1] - point[0]**2)**2
    return point, score

In [ ]:
from dask.distributed import as_completed
from random import uniform

scale = 5                  # Intial random perturbation scale
best_point = (0, 0)        # Initial guess
best_score = float('inf')  # Best score so far

# initial 10 random points
futures = [c.submit(rosenbrock, (uniform(-scale, scale), uniform(-scale, scale))) 
           for _ in range(10)]
iterator = as_completed(futures)

for res in iterator:
    # take a completed point, is it an improvement?
    point, score = res.result()
    if score < best_score:
        best_score, best_point = score, point
        print(score, point)

    x, y = best_point
    
    # add new point, dynamically, to work on the cluster
    new_point = c.submit(rosenbrock, (x + uniform(-scale, scale),
                               y + uniform(-scale, scale)))
    iterator.add(new_point)  # Start tracking new task as well

    # Narrow search and consider stopping
    scale *= 0.99
    if scale < 0.001:
        break
point

In [ ]:
# clean up
del futures[:], new_point, iterator, res